# Earthquake
2024-02-01

This is an example analysis of the Earthquake data for Haiti

Earthquake

In [ ]:
import datetime as dt
import statistics
import pandas as pd
from climada.util.api_client import Client

client = Client()

In [ ]:
earthquake = client.get_hazard("earthquake", properties={
                        "country_iso3alpha": "HTI",
                    })
earthquakes = []
for i, event_intensity in enumerate(earthquake.intensity):
    total_intensity = max(event_intensity.toarray().flatten())
    if total_intensity > 0.0:
        event_date = dt.datetime.fromordinal(earthquake.date[i]).isoformat()[0:10]
        # print(event_date, total_intensity)
        earthquakes.append([event_date, total_intensity])
earthquake_dt = pd.DataFrame(earthquakes)
earthquake_dt.columns= ["date", "total intensity"]
earthquake_dt.plot(y="total intensity",x="date", kind="bar")
print(f"Number of earthquakes:{len(earthquakes)}", flush=True)
average_max_intensity = statistics.mean([x[1] for x in earthquakes]) 
overall_max_intensity = max([x[1] for x in earthquakes])
print(f"Average max intensity: {round(average_max_intensity,2)}", flush=True)
print(f"Overall max intensity: {round(overall_max_intensity,2)}", flush=True)
earthquake.plot_intensity(0)

In [ ]:
import math
def calc_zoom(df):
  width_y = max(df["latitude"]) - min(df["latitude"])
  width_x = max(df["longitude"]) - min(df["longitude"])
  zoom_y = -1.446*math.log(width_y) + 7.2753
  zoom_x = -1.415*math.log(width_x) + 8.7068
  return min(round(zoom_y,2),round(zoom_x,2))

In [ ]:
import plotly.express as px
max_intensity = np.max(earthquake.intensity, axis=0).toarray().flatten()

latitudes = earthquake.centroids.lat
longitudes = earthquake.centroids.lon
country_data = pd.DataFrame({"latitude":latitudes, "longitude":longitudes, "value":max_intensity})
country_data["value"] = (country_data["value"]- min(country_data["value"])) 
zoom = calc_zoom(country_data)
fig = px.scatter_mapbox(country_data, 
                        lat="latitude", lon="longitude",
                        color="value",
                        size="value", size_max=20, 
                        zoom=zoom, opacity=0.75, 
                         mapbox_style='carto-darkmatter',

                         width=1200, height=800)
fig.show()

In [ ]:
from hdx_scraper_climada.climada_interface import calculate_earthquake_timeseries_admin2
from hdx_scraper_climada.download_admin1_geometry import get_admin2_shapes_from_hdx
# earthquakes = calculate_earthquake_timeseries_admin2("Haiti")

admin1_names, admin2_names, admin2_shapes = get_admin2_shapes_from_hdx("HTI")



In [ ]:
# Small change to check nbstripout is working
import geopandas
import pandas
all_shapes = geopandas.GeoDataFrame( pandas.concat(admin2_shapes, ignore_index=True) )
all_shapes["admin1_names"] = admin1_names
all_shapes["admin2_names"] = admin2_names
all_shapes.plot(column = "admin2_names", categorical=True, cmap='Spectral')

In [ ]:
import csv
with open("admin1-timeseries-summaries.csv", encoding="utf-8") as summary_file:
    summary_data = list(csv.DictReader(summary_file))

date_set = set(x["event_date"] for x in summary_data)
print(date_set, flush=True)
values = {x["admin2_name"]:x["value"] for x in summary_data if x["event_date"] == "2010-01-13"}
value_column = [float(values.get(x, 0.0)) for x in admin2_names]
all_shapes = geopandas.GeoDataFrame( pandas.concat(admin2_shapes, ignore_index=True) )
all_shapes["admin1_names"] = admin1_names
all_shapes["admin2_names"] = admin2_names
all_shapes["value"] = value_column
all_shapes.plot(column = "value", cmap='Spectral', legend=True)